In [1]:
import numpy
import time
from tqdm import tqdm
import ipyparallel as ipp
from scipy import stats
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
#from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.providers.aer import AerSimulator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit.quantum_info import Pauli
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, CircuitStateFn 
import scipy
from joblib import Parallel, delayed, Memory
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.compiler import transpile
from qiskit.circuit.random import random_circuit
from qiskit.chemistry.components.variational_forms import UCCSD

/home/amukh/Anaconda3-Linux/lib/python3.8/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')


In [16]:
import warnings
warnings. filterwarnings('ignore')

In [17]:
c =ipp.Client()
num_qubits=8

In [18]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [5]:
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('jordan_wigner')
    return qubit_op

In [6]:
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites] \
            = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
    w,v=numpy.linalg.eigh(Hmat)
    Eg=w[0]
    state_g=v[:,0]
    return qubitH,Hmat,Eg,state_g

In [19]:
%run Utils.ipynb
%run optimizers.ipynb
%run EstimateHamiltonianMean.ipynb

In [8]:
U=7
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)

In [9]:
#Hamiltonian grouping for circuit based measurement
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate,rotH2body=OffDiag2BodyTerms(opArr[2])
#one body off diags
circ0_8gate,rotH1body=OffDiag1BodyTerms(opArr[0])
Ops=[rotH1body,opArr[1],rotH2body]

num. of CXs in 2nd Givens Rotn 2
num. of CXs in 1st Givens Rotn 32


In [10]:
#Control Hamiltonian at lower U value
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U-3)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U-3)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate_ctrl,rotH2body=OffDiag2BodyTerms(opArr[2])
#one body off diags
circ0_8gate_ctrl,rotH1body=OffDiag1BodyTerms(opArr[0])
Ops_ctrl=[rotH1body,opArr[1],rotH2body]

num. of CXs in 2nd Givens Rotn 2
num. of CXs in 1st Givens Rotn 32


In [11]:
#commutator pools
commutator_pool,commutator_pool_2=commutatorPool(qubitH)

In [12]:
U=7
num_sites=4
backend=Aer.get_backend('statevector_simulator')
HFcirc,HFstate,MO_op=HF_init_state(U,return_MO_rotnOp=True)
init_circ = QuantumCircuit(num_qubits)
for i in range(num_sites//2):
    init_circ.x(i)
    init_circ.x(i+num_sites)
var_form_base=UCCSD(8,num_particles=4, initial_state=init_circ,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
var_form_base.manage_hopping_operators()

 33%|███▎      | 1/3 [00:00<00:00,  7.20it/s]

Final Hartree Fock Energy -18.788335371650774


100%|██████████| 3/3 [01:00<00:00, 20.06s/it]


In [20]:
#rotating Hamiltonian to MO rep.
qubitH_MO=MO_Rep_Op(MO_op,qubitH)
Hmat_MO=op_converter.to_matrix_operator(qubitH_MO).dense_matrix

 47%|████▋     | 1923/4096 [00:00<00:00, 8594.98it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

 70%|███████   | 2873/4096 [00:00<00:00, 8842.56it/s]

1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131


100%|██████████| 4096/4096 [00:00<00:00, 10102.52it/s]


4087
4088
4089
4090
4091
4092
4093
4094
4095


 32%|███▏      | 1235/3840 [00:00<00:00, 12321.25it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

100%|██████████| 3840/3840 [00:00<00:00, 11437.18it/s]

2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564


In [34]:
#Hamiltonian for eg Model in MO basis
op= WeightedPauliOperator.from_file('../7/FinalDataFiles/HinMOrep.json')

In [44]:
def commuting_groups(op):
        op_to_pool=[WeightedPauliOperator([item]) for item in op.paulis]
        
        groups=[[op_to_pool[0]]]
        for i in range(1,len(op_to_pool)):
            flag=0
            for j in range(len(groups)):
                check_commute=[op_to_pool[i].commute_with(groups[j][l]) for l in range(len(groups[j]))]
                if False in check_commute:
                    continue
                else:
                    flag=1
                    groups[j].append(op_to_pool[i])
                    break
            if flag==0:
                groups.append([op_to_pool[i]])
        return groups

In [37]:
for i in range(len(ExcOps)):
    var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli.from_label(ExcOps[i])]]))

In [16]:
runs=120
tol=5e-6
qr=QuantumRegister(8,'q')

In [39]:
#params=[]
EnergyArr=[]
numCX=0
CXarr=[]
#ExcOps=[]
error=1000
steps=33
stepNo=7
timeArr=[]
t1=time.time()
num_shots_arr=[2**14,2**15,2**15]
while error>1e-5 and stepNo<steps:
    PauliOp=ansatzExpander(ordChoice='1')
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("excitationOp",ExcOps[-1])
    with open('../7/labels.txt','+a') as f:
        print(ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    #construct circuit for estimating Hamiltonian
    circ=var_form_base.construct_circuit(parameters=params)
    params,E=AdaDelta_qasm(HamiltonianEstmFrugal,params,num_shots_arr=num_shots_arr,runs=runs,tol=tol,save_opt_steps=True)
#    params,E=AdaDelta_sv(Energy,params,runs=runs,tol=tol,save_opt_steps=True)
#    params,E=SMO_qasm(HamiltonianEstmFrugal,params,runs=runs,tol=tol,save_opt_steps=True) 
#    params,E=AdaDeltaSMO_qasm(HamiltonianEstmFrugal,params,num_shots_arr=[2**8,2**9,2**9],runs=runs,tol=tol,save_opt_steps=True)
    params=list(params)
    numCX=numCX+countCXs(ExcOps[-1])
    CXarr.append(numCX)
    EnergyArr.append(E)
    timeArr.append(time.time()-t1)
    error=E-Eg
    print("convergence energy",EnergyArr[-1])
    
    print("time elapsed",timeArr[-1])
    stepNo=stepNo+1
    
    

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:   44.5s


max. gradient 0.22576366559004002
excitationOp IIXYIIXX


[Parallel(n_jobs=7)]: Done  56 out of  56 | elapsed:   44.8s finished


index 0
updated energy -19.154552229039176 learning rate [0.99659413+0.j 0.47029897+0.j 0.45183394+0.j 0.86246082+0.j
 0.47974966+0.j 0.74435477+0.j 0.64643775+0.j 0.57159255+0.j
 0.55276399+0.j 0.49539083+0.j 0.95122356+0.j 0.61988805+0.j
 0.31241228+0.j 0.25782368+0.j 0.15121384+0.j]
error 0.7516566981132087
index 1
updated energy -19.155262192597526 learning rate [0.9938273 +0.j 0.55883132+0.j 0.16679272+0.j 0.43671846+0.j
 0.37886232+0.j 0.13090269+0.j 0.37429572+0.j 0.21289793+0.j
 0.58051849+0.j 0.28721481+0.j 0.29644924+0.j 0.15366491+0.j
 0.45055845+0.j 0.23641303+0.j 0.1894349 +0.j]
error 0.563920014474494
index 2
updated energy -19.146473018696707 learning rate [0.86046842+0.j 0.61463919+0.j 0.1624028 +0.j 0.52612963+0.j
 0.48532564+0.j 0.18895372+0.j 0.12473528+0.j 0.28503951+0.j
 0.24107772+0.j 0.25818063+0.j 0.31569198+0.j 0.20843747+0.j
 0.18339322+0.j 0.15019936+0.j 0.22052842+0.j]
error 0.4251373043310753
index 3
updated energy -19.14293097558957 learning rate [0.834717

KeyboardInterrupt: 

In [ ]:
# E_arr=[]
# E_arr2=[]
# E_arr3=[]
# num_shots_arr=[2**8,2**9,2**9]
# string_shots_label=','.join([str(shots) for shots in num_shots_arr])
# with open('../'+str(U)+'/adaDeltaOptStepsWithQasm'+string_shots_label+'.txt','r') as f:
#     lines=f.readlines()
#     for line in lines:
#         elems=line.split('#')
#         E_arr.append(float(elems[1]))
# num_shots_arr=[2**4,2**5,2**5]
# string_shots_label=','.join([str(shots) for shots in num_shots_arr])
# with open('../'+str(U)+'/adaDeltaOptStepsWithQasm'+string_shots_label+'.txt','r') as f:
#     lines=f.readlines()
#     for line in lines:
#         elems=line.split('#')
#         E_arr2.append(float(elems[1]))
# num_shots_arr=[2**5,2**6,2**6]
# string_shots_label=','.join([str(shots) for shots in num_shots_arr])
# with open('../'+str(U)+'/adaDeltaOptStepsWithQasm'+string_shots_label+'.txt','r') as f:
#     lines=f.readlines()
#     for line in lines:
#         elems=line.split('#')
#         E_arr3.append(float(elems[1]))        

In [ ]:
# import matplotlib.pyplot as plt
# figure,ax=plt.subplots(1,figsize=(10,7))
# ax.set_yscale('log')
# ax.tick_params(labelsize=15)
# ax.plot(numpy.array(E_arr[:363])+19.0701686065437848,label='shots=2^8,2^9,2^9',marker='o')
# ax.set_xlabel('runs')
# ax.legend(fontsize=15)
# ax.set_ylabel('abs. Error')